# Cleaning the datasets for Registro Público de Concesiones and zipcodes

This code includes all the cleaning done for the datasets that were scrapped and aggregates these into a big dataset.

- [Zipcodes](https://xn--cdigospostales-lob.es/listado-de-codigos-postales-de-espana/)
- [Main dataset](https://sedeaplicaciones.minetur.gob.es/RPC_Consulta)
    - Main page dataset
    - Pop-up datasets

## Packages used

- datetime
- numpy
- pandas
- fuzzywuzzy
- math
- datetime

# Import packages

In [128]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime as dt
import math

# Import data

In [129]:
# Main dataset
main_df = pd.read_csv('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/RegistroPublicoConcesiones_General.csv')
# Zipcodes dataset
zipcodes = pd.read_excel('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/new_data/listado-codigos-postales-con-LatyLon.xls')
# Pop-up
df = pd.read_csv('/Users/niko/Downloads/RegistroPublicoConcesiones_Frequencies.csv',sep=';')

# Import all the datasets for all pop-ups
# region_1 = pd.read_csv('')
# region_2 = pd.read_csv('')
# region_3 = pd.read_csv('')
# region_4 = pd.read_csv('')
# region_5 = pd.read_csv('')
# region_6 = pd.read_csv('')
# region_7 = pd.read_csv('')
# region_8 = pd.read_csv('')
# region_9 = pd.read_csv('')
# region_10 = pd.read_csv('')
# region_11 = pd.read_csv('')
# region_12 = pd.read_csv('')
# region_13 = pd.read_csv('')
# region_14 = pd.read_csv('')
# region_15 = pd.read_csv('')
# region_16 = pd.read_csv('')
# region_17 = pd.read_csv('')
# region_18 = pd.read_csv('')
# region_19 = pd.read_csv('')

# Concat all the datasets
# df = pd.concat([
#                         region_1,
#                         region_2,
#                         region_3,
#                         region_4,
#                         region_5,
#                         region_6,
#                         region_7,
#                         region_8,
#                         region_9,
#                         region_10,
#                         region_11,
#                         region_12,
#                         region_13,
#                         region_14,
#                         region_15,
#                         region_16,
#                         region_17,
#                         region_18,
#                         region_19,
#                         ])

# Clean the datasets

## Clean the pop-up data
Doing a fuzzy join on the dataset of zipcodes and the data of reference <br>
data inside the 'Consulta del Registro Público de Concesiones'.
This is needed, because the data from the source does not have zipcodes, longitute and latitude.<br>
The fuzzy join does a cross join, then calculates the fuzzy ratio and include the data based on a fuzzy_ratio.<br>
Then, the duplicated columns are dropped.<br>
The output is a new dataset that includes the zipcode, longitude and latitude per tower.

In [130]:
# df.head(50)

In [131]:
# Clean all the pop-up datasets
# df = df.drop(df.columns[[0]],axis = 1)
# df = df.drop(df.columns[[0]],axis = 1)
df = df.apply(lambda x: x.str.strip()).replace('', np.nan)
df = df.fillna(method='ffill')
df[['Frequencias', 'Tipo']] = df['Frecuencia'].str.split(' ', 1, expand=True)
del df['Frecuencia']
df['Frequencias'] = df['Frequencias'].apply(lambda x: x.replace('.', '')).apply(lambda x: x.replace(',', '.')).astype('float')

### Cleaning of zipcodes

In [132]:
# zipcodes.head(50)

In [133]:
# Define the columns that we want to include in the final dataset
zipcodes_columns = ['codigopostalid','lat','lon']
df_columns = [ 'Referencia','Comunidad','Provincia','Municipio','Frequencias','Tipo']
all_new_columns = df_columns+zipcodes_columns

### Join pop-ups and zipcodes

In [134]:
# Define the fuzzy ratio used to include the data
fuzzy_ratio = 20

# Create new column to use to join both datasets
zipcodes['merge']='all'
df['merge']='all'

# Join both datasets per row
all_datasets = pd.merge(df,zipcodes,on='merge')
del all_datasets['merge']

# Create list of tuples based on the columns that we want to use for the join
datasets_tuple = all_datasets[['Municipio', 'poblacion']].apply(tuple, axis=1).tolist()

# Create the fuzz ratio on the list of tuples ceated
all_datasets['ratio'] = [fuzz.token_sort_ratio(*i) for i in datasets_tuple]

# Exclude those that have a low match ratio, the threshhold is set low because some matches have a low score
all_datasets = all_datasets[all_datasets.ratio>fuzzy_ratio]

# Drop all duplicates based on the defined columns and keep all the wanted ones
final_df = all_datasets[all_new_columns].drop_duplicates(subset=['Referencia','Municipio','Frequencias'])

In [135]:
# Intermediate step to save the data
final_df.to_csv('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/popup_zipcodes.csv',sep=';')

In [136]:
# final_df.head()

In [137]:
# final_df.tail()

## Clean the main dataset


In [138]:
# Remove the rows that are unecessary
# Function to return a list with unique numeric values
def unique(list1):
    x = np.array(list1)
    return list(np.unique(x))

# Creates list of the returned values 
list_cities = unique(main_df.Localidad)

# Includes only text values, all cities of Spain
new_list_cities = []
for i in list_cities:
    if i.isnumeric() is False:
        if i != ' ':
            new_list_cities.append(i)

# Filter out all the values that are not inside the new_list_cities
main_df = main_df.loc[main_df['Localidad'].isin(new_list_cities)]

# --- Work on booleans

# Fill in False to all nulls for specific boolean columns
main_df[['Susceptible cesion','Susceptible mutualizacion','Obtenido por transferencia']] = \
                            main_df[['Susceptible cesion','Susceptible mutualizacion','Obtenido por transferencia']].fillna(False)
                            
main_df['Susceptible cesion'] = main_df['Susceptible cesion'].replace("true", True)
main_df['Obtenido por transferencia'] = main_df['Obtenido por transferencia'].replace("Detalle", True)

# --- Work on the dates

# Select columns that contain dates
date_columns = ['F. Caducidad','F. Concesion']

# Transform date objects to datetime
main_df[date_columns] = main_df[date_columns].apply(pd.to_datetime, errors='coerce',infer_datetime_format=True)

# New features day, month and year
main_df['dia_concesion'] = round((main_df['F. Caducidad'] - main_df['F. Concesion']).dt.days,0).fillna(0).apply(np.int64)
main_df['mes_concesion'] = round((main_df['F. Caducidad'] - main_df['F. Concesion']).dt.days/12).fillna(0).apply(np.int64)
main_df['año_concesion'] = round((main_df['F. Caducidad'] - main_df['F. Concesion']).dt.days/360).fillna(0).apply(np.int64)

## Joined datasets
Joining both datasets to create the final dataset that needs to be standarized.

In [139]:
df_joined = pd.merge(main_df,final_df, how='left', left_on = 'Referencia', right_on = 'Referencia')

In [140]:
# Intermediate step to clean the data
df_joined.to_csv('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/dj_joined.csv',sep=';')

### Total shape of datasets

In [141]:
print(f'Pop-up and zipcodes.\n Columns: {final_df.shape[1]} & Rows: {final_df.shape[0]}\n')
print(f'Main dataset.\n Columns: {main_df.shape[1]} & Rows: {main_df.shape[0]}\n')
print(f'Final dataset with zipcodes, pop-ups and the main page.\n Columns: {df_joined.shape[1]} & Rows: {df_joined.shape[0]}\n')

total_rows = final_df.shape[0]+main_df.shape[0]
final_total_rows = df_joined.shape[0]
absolute_values = total_rows-final_total_rows
percentage_final_rows = round(1-final_total_rows/total_rows,4)*100


print(f'Total % loss of rows of the final dataset: {percentage_final_rows}\nTotal absolute loss of rows of the final dataset: {absolute_values}')

Pop-up and zipcodes.
 Columns: 9 & Rows: 3133

Main dataset.
 Columns: 15 & Rows: 5209

Final dataset with zipcodes, pop-ups and the main page.
 Columns: 23 & Rows: 7554

Total % loss of rows of the final dataset: 9.45
Total absolute loss of rows of the final dataset: 788


# Standarization of both datasets

## Clean of 'Titular'
Try to remove as many duplicated entries and wrongly entered data and identify unique vendors.<br>
We have entries that include for example 'Vodafone Ono' and 'Vodafone Espagna'. While these companies are not the same, they are from the same mother company.<br>
We needed to re-join the dataset and due to the fuzzy join we did to clean this data, we lost some rows. This cumulates to approximately 13.68% of the data.

In [142]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7554 entries, 0 to 7553
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Referencia                  7554 non-null   object        
 1   Titular                     7554 non-null   object        
 2   NIF/CIF                     7554 non-null   object        
 3   Domicilio social            7554 non-null   object        
 4   Localidad                   7554 non-null   object        
 5   Provincia_x                 7554 non-null   object        
 6   C. Postal                   7554 non-null   int64         
 7   F. Concesion                2966 non-null   datetime64[ns]
 8   F. Caducidad                7553 non-null   datetime64[ns]
 9   Susceptible cesion          7554 non-null   bool          
 10  Susceptible mutualizacion   7554 non-null   bool          
 11  Obtenido por transferencia  7554 non-null   bool        

In [143]:
# Check if nulls in dataset
# [print(i) for i in df_joined['Titular'].isna() if i == True]

In [144]:
# for col in df_joined[['Titular']]:
#     df_joined[col] = df_joined[col].str.strip()
#     print('Number of unique values in ' + str(col) +': ' + str(df[col].nunique()))

In [145]:
# Find unique owners
unique_titular = df_joined['Titular'].unique().tolist()

#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in unique_titular 
             for i in process.extract(x, unique_titular,     scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])

similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])


high_score_sort =  \
similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['brand_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

grouped_high_score = high_score_sort.groupby(['brand_sort','score_sort'],as_index=False).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

                        # Create new column to use to join both datasets
fuzzy_ratio_final = 60

grouped_high_score['merge']='all'
df_joined['merge']='all'

# Join both datasets per row
all_datasets = pd.merge(df_joined,grouped_high_score,on='merge')
del all_datasets['merge']

# Create list of tuples based on the columns that we want to use for the join
datasets_tuple = all_datasets[['Titular', 'match_sort']].apply(tuple, axis=1).tolist()

# Create the fuzz ratio on the list of tuples ceated
all_datasets['ratio'] = [fuzz.token_sort_ratio(*i) for i in datasets_tuple]

# Exclude those that have a low match ratio, the threshhold is set low because some matches have a low score
all_datasets = all_datasets[all_datasets.ratio>fuzzy_ratio_final]

# Drop all duplicates based on the defined columns and keep all the wanted ones
df_re_joined = all_datasets.drop_duplicates(subset=['Referencia'])

In [148]:
# Intermediate step to clean the data
df_re_joined.to_csv('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/final_cleansed_standarized.csv',sep=';')

In [147]:
print('First transformation\n','--'*30)

print(f'Pop-up and zipcodes.\n Columns: {final_df.shape[1]} & Rows: {final_df.shape[0]}\n')
print(f'Main dataset.\n Columns: {main_df.shape[1]} & Rows: {main_df.shape[0]}\n')
print(f'Final dataset with zipcodes, pop-ups and the main page.\n Columns: {df_joined.shape[1]} & Rows: {df_joined.shape[0]}\n')

total_rows = final_df.shape[0]+main_df.shape[0]
final_total_rows = df_joined.shape[0]
absolute_values = total_rows-final_total_rows
percentage_final_rows = round(1-final_total_rows/total_rows,4)*100

print(f'Total % loss of rows of the final dataset: {percentage_final_rows}\nTotal absolute loss of rows of the final dataset: {absolute_values}\n')

print('Last transformation\n','--'*30)
print(f'Columns: {df_joined.shape[1]} & Rows: {df_joined.shape[0]}\n')

absolute_loss_from_original_df = main_df.shape[0] - df_re_joined.shape[0]
total_loss_from_original_df = round(1-df_re_joined.shape[0]/main_df.shape[0],4)*100

print('Comparison with original dataset')
print(f"Total loss of originaldata when joining back at datasets with current threshhold ({fuzzy_ratio_final}): {total_loss_from_original_df}\nTotal absolute loss of rows of the final dataset: {absolute_loss_from_original_df}\n")
absolute_loss_from_cleansed_df = df_joined.shape[0] - df_re_joined.shape[0]
total_loss_from_cleansed_df = round(1-df_re_joined.shape[0]/df_joined.shape[0],4)*100
print('\nComparison with final cleansed dataset')
print(f'Total loss of originaldata when joining back at datasets with current threshhold ({fuzzy_ratio_final}): {total_loss_from_cleansed_df}\nTotal absolute loss of rows of the final dataset: {absolute_loss_from_cleansed_df}\n')

First transformation
 ------------------------------------------------------------
Pop-up and zipcodes.
 Columns: 9 & Rows: 3133

Main dataset.
 Columns: 15 & Rows: 5209

Final dataset with zipcodes, pop-ups and the main page.
 Columns: 24 & Rows: 7554

Total % loss of rows of the final dataset: 9.45
Total absolute loss of rows of the final dataset: 788

Last transformation
 ------------------------------------------------------------
Columns: 24 & Rows: 7554

Comparison with original dataset
Total loss of originaldata when joining back at datasets with current threshhold (60): 4.9
Total absolute loss of rows of the final dataset: 255


Comparison with final cleansed dataset
Total loss of originaldata when joining back at datasets with current threshhold (60): 34.42
Total absolute loss of rows of the final dataset: 2600

